In [ ]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
zip_dir = tf.keras.utils.get_file('cats_and_dogs_filterted.zip', origin=_URL, extract=True)

In [ ]:
zip_dir_base = os.path.dirname(zip_di r)
!find $zip_dir_base -type d -print

/root/.keras/datasets
/root/.keras/datasets/cats_and_dogs_filtered
/root/.keras/datasets/cats_and_dogs_filtered/train
/root/.keras/datasets/cats_and_dogs_filtered/train/dogs
/root/.keras/datasets/cats_and_dogs_filtered/train/cats
/root/.keras/datasets/cats_and_dogs_filtered/validation
/root/.keras/datasets/cats_and_dogs_filtered/validation/dogs
/root/.keras/datasets/cats_and_dogs_filtered/validation/cats


In [ ]:
list_ds =  tf.data.Dataset.list_files(train_dir + '*/*/*', shuffle=True)
print(list_ds)*


<ShuffleDataset shapes: (), types: tf.string>


In [ ]:
class_names  = ['dogs', 'cats']
class_names = tf.Variable(class_names)

In [ ]:
val_size = int(len(list_ds) * 0.2)
train_ds = list_ds.skip(val_size)

val_ds = list_ds.take(val_size)


In [ ]:
print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())

1600
400


In [ ]:
def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)
  label = tf.argmax(parts[-2] == class_names)
  return label  

In [ ]:
def decode_img(img):
  img = tf.image.decode_jpeg(img, channels=3)
  return tf.image.resize(img, [180,180])

In [ ]:
def process_path(file_path):
  label = get_label(file_path)
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label 
  

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls= AUTOTUNE)

In [ ]:
batch_size = 32
def configure(ds):
  ds = ds.cache()
  ds = ds.shuffle(buffer_size = 1000)
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size = AUTOTUNE)
  return ds 

train_ds = configure(train_ds)
val_ds = configure(val_ds)


In [ ]:
def base_model():
  inputs = tf.keras.Input(shape = (180,180,3))
  x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
  x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D()(x)
  x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D()(x)
  x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D()(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
  model = tf.keras.Model(inputs= inputs, outputs=outputs)
  return model  
  
model = base_model()

In [ ]:

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.BinaryCrossentropy()
train_acc_metric = tf.keras.metrics.BinaryAccuracy()
val_acc_metric = tf.keras.metrics.BinaryAccuracy()



In [ ]:
def apply_gradient(optimizer, model, x, y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss_value = loss_object(y_true = tf.reshape(y, (-1, 1)), y_pred = logits)
    #print(loss_value)
  gradients = tape.gradient(loss_value, model.trainable_weights)

  optimizer.apply_gradients(zip(gradients, model.trainable_weights))
  return logits, loss_value 

In [ ]:

def train_epoch(): 
    
  losses = []
  for step, (x_batch_train, y_batch_train) in enumerate(train_ds):
    logits, loss_value = apply_gradient(optimizer, model,x_batch_train, y_batch_train)
    losses.append(loss_value)
    train_acc_metric(y_batch_train, logits)

    return losses 


In [ ]:


def perform_validation():
  losses = []
  for x_val, y_val in val_ds:

    val_logits = model(x_val)
    val_loss = loss_object(y_true=y_val, y_pred=val_logits)
    #print('val_loss', val_loss)
    losses.append(val_loss)
    val_acc_metric(y_val, val_logits)
    return losses

In [ ]:


epochs = 100
epochs_val_losses = []
epochs_trian_losses =[] 
for epoch in range(epochs):
    print('Start of epoch %d' %(epoch,))
    
    losses_train = train_epoch()
    train_acc = train_acc_metric.result()

    losses_val = perform_validation()
    val_acc = val_acc_metric.result()
    # print(train_acc)
    # print(val_acc)
    loss_train_mean = np.mean(losses_train)
    loss_val_mean = np.mean(losses_val)
    #print(loss_val_mean)
    epochs_val_losses.append(loss_val_mean)
    epochs_trian_losses.append(loss_train_mean)

    print('\n Epoch %s: Train loss: %.4f  Validation Loss: %.4f, \
    Train Accuracy: %.4f, Validation Accuracy %.4f' \
        % (epoch, float(loss_train_mean), float(loss_val_mean), 
        float(train_acc), float(val_acc)))

    train_acc_metric.reset_states()
    val_acc_metric.reset_states()

In [ ]:
print(model.trainable_weights)

In [ ]:
for _, (x,y) in enumerate(train_ds):
  print(tf.shape(x))

In [ ]:
model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 180, 180, 3)]     0         
_________________________________________________________________
rescaling_14 (Rescaling)     (None, 180, 180, 3)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 178, 178, 32)      896       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 87, 87, 32)        9248      
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 43, 43, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 41, 41, 32)        924

In [ ]:
num_classes = 5

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds, 
  validation_data=val_ds,
  epochs=3
)

Epoch 1/3
50/50 [==============================] - 58s 1s/step - loss: 0.8792 - accuracy: 0.4546 - val_loss: 0.6756 - val_accuracy: 0.5975
Epoch 2/3
50/50 [==============================] - 57s 1s/step - loss: 0.6838 - accuracy: 0.5490 - val_loss: 0.6346 - val_accuracy: 0.6500
Epoch 3/3
50/50 [==============================] - 57s 1s/step - loss: 0.5890 - accuracy: 0.7011 - val_loss: 0.5254 - val_accuracy: 0.7275
